# Index

1. [Imports](#imports)
2. [Parabolic Function - Derivative example](#parabolic-function---derivative-example)
3. [Forward Pass Manually](#forward-pass-manually)
4. [First Manual backpropagation](#first-manual-backpropagation)
5. [Hiperbolic tangent definition](#hiperbolic-tangent-definition)
6. [Second Manual backpropagation](#second-manual-backpropagation)
    - [Initialize values of the network](#initialize-values-of-the-network)
    - [Backpropagation](#backpropagation)
7. [Semiautomation of the backpropagation](#semiautomation-of-the-backpropagation)
    - [Reset values of the network](#reset-values-of-the-network)
8. [Topological graph](#topological-graph)
    - [Explanation](#explanation)
9. [Automation of the backpropagation](#automation-of-the-backpropagation)
10. [New topologies - Autonomous Backpropagation](#new-topologies---Autonomous-Backpropagation)
11. [Pytorch approach](#pytorch-approach)

## Imports

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
from src.backpropagation import *
from src.graph_nn import *
from src.nn import *
%load_ext autoreload
%autoreload 2

In [ ]:
f(2)

## Parabolic Function - Derivative example

In [ ]:
xs = np.arange(-5,5,0.25)
ys = f(xs)
plt.plot(xs,ys)
plt.grid()

In [ ]:
h = 0.0000001
x = -3.0
(f(x+h) - f(x))/h

## Forward Pass Manually

In [ ]:
# lets get more complex
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
print(d)

In [ ]:
h = 0.0000001

a = 2.0
b = -3.0
c = 10.0


d1 = a*b + c
a += h
d2 = a*b + c

print('d1',d1)
print('d2',d2)
print('slope',(d2-d1)/h)

In [ ]:
h = 0.0000001

a = 2.0
b = -3.0
c = 10.0


d1 = a*b + c
b += h
d2 = a*b + c

print('d1',d1)
print('d2',d2)
print('slope',(d2-d1)/h)

In [ ]:
a+b

In [10]:
d = a*b+c

Initial values

In [11]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d*f; L.label = 'L'

Manual backward propagation

## First Manual backpropagation

In [ ]:
draw_dot(L)

L = L

dL/dL = 1

In [13]:
L.grad = 1.0

In [ ]:
draw_dot(L)

$$
\begin{align*}
L &= d \cdot f \\
\frac{dL}{dd} &= f \\
\frac{f(x+h) + f(a)}{h} &= \frac{(d+h) \cdot f - d \cdot f}{h} \\
&= \frac{d \cdot f + h \cdot f - d \cdot f}{h} \\
&= \frac{h \cdot f}{h} \\
&= f
\end{align*}
$$


In [15]:
f.grad = d.data
d.grad = f.data

In [ ]:
draw_dot(L)

$$
\begin{align*}
\frac{dL}{dc} &= ? \\
\frac{dL}{de} &= ? \\
\frac{dd}{dc} &= ? \\
d &= e + c \\
\frac{dd}{dc} &= 1 \\
\frac{dd}{de} &= 1 \\
\frac{dL}{dc} &= \frac{dL}{dd} \cdot \frac{dd}{dc}
\end{align*}
$$


In [17]:
e.grad = d.grad  * 1
c.grad = d.grad  * 1

The plus nodes works as a routing for the gradient at the backpropagation

In [ ]:
draw_dot(L)

$$
\begin{align*}
\frac{dL}{da} &= ? \\
\frac{dL}{db} &= ? \\

e &= a \cdot b \\
\frac{de}{da} &= b \\
\frac{de}{db} &= a \\
\frac{dL}{da} &= \frac{dL}{dd} \cdot \frac{dd}{de} \cdot \frac{de}{da} = f \cdot 1 \cdot b \\
\frac{dL}{db} &= \frac{dL}{dd} \cdot \frac{dd}{de} \cdot \frac{de}{db} = f \cdot 1 \cdot a
\end{align*}
$$


In [19]:
a.grad = e.grad  * 1 * b.data
b.grad = e.grad  * 1 * a.data

In [ ]:
draw_dot(L)

Manual forward pass

In [ ]:
a.data += a.grad * 0.01
b.data += b.grad * 0.01
c.data += c.grad * 0.01
f.data += f.grad * 0.01

e = a*b
d = e + c
L = d*f

print(L.data)

In [ ]:
def lol():

    h = 0.001

    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label='f')
    L = d*f; L.label = 'L'
    L1 = L.data

    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0+h, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    # d.data += h
    f = Value(-2.0, label='f')
    L = d*f; L.label = 'L'
    L2 = L.data

    print('dL/dL', (L2-L1)/h)

lol()

## Hiperbolic tangent definition

In [ ]:
plt.plot(xs, np.tanh(xs))
plt.grid()

## Second Manual backpropagation

### Initialize values of the network

In [ ]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias b
b = Value(6.8813735870195432, label='b')

# x1w1 + x2w2 + b
x1w1 = x1 * w1
x1w1.label = 'x1*w1'

x2w2 = x2 * w2
x2w2.label = 'x2*w2'

x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1*w1 + x2*w2'

n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
draw_dot(o)

## Backpropagation

In [25]:
o.grad = 1.0

In [ ]:
draw_dot(o)

do / dn

o = tanh(n)

do = 1 - tanh(n)**2 = 1 o**2

In [27]:
n.grad = (1 - o.data**2)

In [ ]:
draw_dot(o)

In [29]:
b.grad = n.grad * 1
x1w1x2w2.grad = n.grad * 1

In [ ]:
draw_dot(o)

In [31]:
x1w1.grad = x1w1x2w2.grad * 1
x2w2.grad = x1w1x2w2.grad * 1

In [ ]:
draw_dot(o)

x1w1 = x1 * w1

x2w2 = x2 * w2

dx1w1 / dx1 = x2

dx2w2 / dx2 = x1

In [33]:
x1.grad = x1w1.grad * w1.data
w1.grad = x1w1.grad * x1.data

In [ ]:
draw_dot(o)

In [35]:
x2.grad = x2w2.grad * w2.data
w2.grad = x2w2.grad * x2.data

In [ ]:
draw_dot(o)

## Semiautomation of the backpropagation

### Reset values of the network

In [37]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias b
b = Value(6.8813735870195432, label='b')

# x1w1 + x2w2 + b
x1w1 = x1 * w1
x1w1.label = 'x1*w1'

x2w2 = x2 * w2
x2w2.label = 'x2*w2'

x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1*w1 + x2*w2'

n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'

In [ ]:
draw_dot(o)

In [ ]:
o.grad = 1.0
o._backward()
draw_dot(o)

In [ ]:
n._backward()
b._backward()
draw_dot(o)

In [ ]:
x1w1x2w2._backward()
draw_dot(o)

In [ ]:
x2w2._backward()
x1w1._backward()
draw_dot(o)

## Topological graph

In [ ]:
topo = []
visited = set()

def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)

build_topo(o)
topo

### Explanation

This code defines a topological sorting function, which is useful when dealing with directed graphs (like computational graphs in deep learning). Let's go step by step:

1. **`topo = []`**: 
   - This is an empty list that will eventually store the vertices of the graph in topological order (a linear ordering of vertices where for every directed edge `u -> v`, `u` comes before `v`).

2. **`visited = set()`**: 
   - `visited` is a set used to keep track of all nodes that have already been processed (visited). This avoids revisiting the same node and potentially falling into infinite loops when dealing with graphs that have cycles.
   - The `set()` function creates an empty set. A set is a data structure that stores unique elements and allows fast membership testing. So, when you check `if v not in visited`, it’s an efficient way to see if `v` has been processed before.
   
   **Why `set()`?**  
   - The use of `set()` helps ensure that nodes are only visited once because a set automatically handles duplicates (i.e., it only stores unique elements).
   - The method `visited.add(v)` adds the node `v` to the set, ensuring it won't be visited again.

3. **`build_topo(v)`**:
   - This is a recursive function that processes each vertex `v`.
   - It checks whether `v` has already been visited using the `visited` set. If not, it adds `v` to the `visited` set and processes all its children recursively using `for child in v._prev: build_topo(child)`.
   - Once all children of `v` are processed, `v` is added to the `topo` list.

4. **`build_topo(0)`**:
   - This line starts the recursive topological sorting with the vertex `0` (the starting node). It assumes that `v._prev` contains a list of preceding nodes connected to `v`.

5. **`topo`**:
   - After the recursion completes, `topo` will contain the nodes in topologically sorted order.

### Example Use Case:
This pattern is often used in dependency resolution, where you want to process nodes in a directed acyclic graph (DAG) in such a way that each node is processed after all its dependencies have been processed. For instance, in deep learning, it might be used to ensure that you compute gradients in the correct order in the backward pass.

## Automation of the backpropagation

Reseting gradients

In [44]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias b
b = Value(6.8813735870195432, label='b')

# x1w1 + x2w2 + b
x1w1 = x1 * w1
x1w1.label = 'x1*w1'

x2w2 = x2 * w2
x2w2.label = 'x2*w2'

x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1*w1 + x2*w2'

n = x1w1x2w2 + b
n.label = 'n'
o = n.tanh(); o.label = 'o'

In [ ]:
draw_dot(o)

In [46]:
# o._prev, n._prev

In [ ]:
o.backward()
draw_dot(o)

## New topologies - Autonomous Backpropagation

In [48]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias b
b = Value(6.8813735870195432, label='b')

# x1w1 + x2w2 + b
x1w1 = x1 * w1
x1w1.label = 'x1*w1'

x2w2 = x2 * w2
x2w2.label = 'x2*w2'

x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1*w1 + x2*w2'

n = x1w1x2w2 + b
n.label = 'n'
o = n.tanh(); o.label = 'o'

In [ ]:
draw_dot(o)

In [ ]:
o.backward()
draw_dot(o)

In [ ]:
a = Value(2.0, label='a')
b = a + a ; b.label = 'b'
b.backward()
draw_dot(b)

In [ ]:
a = Value(-2.0, label='a')
b = Value(3.0, label='b')

d = a * b
d.label = 'd'
e = a + b
e.label = 'e'
f = d * e
f.label = 'f'

f.backward()
draw_dot(f)


In [ ]:
z = Value(1.0, label='z')
tanh_z = z.tanh()  # Call the tanh method of the Value class
tanh_z

## Pytorch approach

In [54]:
import torch

In [ ]:
tensor = torch.Tensor([[1,2,3],[4,5,6]])
tensor

In [ ]:
tensor.shape

In [ ]:
# Input of the network
x1 = torch.Tensor([2.0]).double()           ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()           ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()          ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()           ; w2.requires_grad = True
b  = torch.Tensor([6.8813735870195432]).double() ; b.requires_grad = True

# Forward pass
n = x1*w1 + x2*w2 + b
# Activation function
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('----')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

In [ ]:
o

In [ ]:
o.item()

In [ ]:
x = [2.0, 3.0, -1]
n = Neuron(len(x))
n(x)

In [ ]:
layers = Layer(len(x), 3)
layers(x)

In [ ]:
n = MLP(len(x), [4,4,1])
n(x)

In [ ]:
draw_dot(n(x))

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0]  # desired targets
y_pred = [n(x) for x in xs]
y_pred

In [ ]:
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, y_pred)])
loss

In [ ]:
loss.backward()